## Step1: Test gym library

In [1]:
import gym
env = gym.make("LunarLander-v2", render_mode="human")
env.action_space.seed(42)

observation, info = env.reset(seed=42, return_info=True)

for _ in range(1000):
    observation, reward, done, info = env.step(env.action_space.sample())

    if done:
        observation, info = env.reset(return_info=True)

env.close()

/Users/strivin/gym/gym/core.py:329: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/Users/strivin/gym/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


## Step2: Construct GridTreasure Environment

In [1]:
import gym
from gym import spaces
import numpy as np
from typing import Optional
from gym.utils.renderer import Renderer

$\Rightarrow 游戏描述:$

$在一个size\times size的网格世界中，有一个蓝色的智能小球，它能在网格世界中上下左右移动，同时也有一个红色小方格，上面放着宝藏$

$蓝色小球和红色方格的初始位置都是在游戏开始前随机确定的，求解蓝色小球寻找宝藏的最优策略$

In [29]:
class GridTreasureEnv(gym.Env):
    metadata = {"render_modes": ["human", "rgb_array", "single_rgb_array","hide"], "render_fps": 4}

    def __init__(self, render_mode: Optional[str] = None, size: int = 5):
        assert render_mode is None or render_mode in self.metadata["render_modes"]
        self.render_mode = render_mode  # Define the attribute render_mode in your environment

        self.size = size  # The size of the square grid
        self.window_size = 512  # The size of the PyGame window
        
        self.numS = size * size
        self.numA = 4

        # Observations are dictionaries with the agent's and the target's location.
        # Each location is encoded as an element of {0, ..., `size`}^2, i.e. MultiDiscrete([size, size]).
        self.observation_space = spaces.Dict(
            {
                "agent": spaces.Box(0, size - 1, shape=(2,), dtype=int),
                "target": spaces.Box(0, size - 1, shape=(2,), dtype=int),
            }
        )

        # We have 4 actions, corresponding to "right", "up", "left", "down"
        self.action_space = spaces.Discrete(self.numA)

        """
        The following dictionary maps abstract actions from `self.action_space` to 
        the direction we will walk in if that action is taken.
        I.e. 0 corresponds to "right", 1 to "up" etc.
        """
        self._action_to_direction = {
            0: np.array([1, 0]), # right
            1: np.array([0, 1]), # up
            2: np.array([-1, 0]),# left
            3: np.array([0, -1]),# down
        }

        """
        If human-rendering is used, `self.window` will be a reference
        to the window that we draw to. `self.clock` will be a clock that is used
        to ensure that the environment is rendered at the correct framerate in
        human-mode.
        """
        if self.render_mode == "human":
            import pygame  # import here to avoid pygame dependency with no render

            pygame.init()
            pygame.display.init()
            self.window = pygame.display.set_mode((self.window_size, self.window_size))
            self.clock = pygame.time.Clock()
                
        # The following line uses the util class Renderer to gather a collection of frames 
        # using a method that computes a single frame. We will define _render_frame below.
        self.renderer = Renderer(self.render_mode, self._render_frame)
        
    def _get_obs(self):
        return {"agent": self._agent_location, "target": self._target_location}
    
    def _get_info(self):
        return {"distance": np.linalg.norm(self._agent_location - self._target_location, ord=1)}
    
    def loc_to_state(self,loc): # mapping location to state
        return loc[0] + loc[1]*self.size
    
    def state_to_loc(self,state): # mapping state to location
        return np.array(
            [state%self.size,state//self.size],dtype=np.int
        )
    
    def reset(self, seed=None, return_info=False, options=None):
        # We need the following line to seed self.np_random
        super().reset(seed=seed)

        # Choose the agent's location uniformly at random
        self._agent_location = self.np_random.integers(0, self.size, size=2)

        # We will sample the target's location randomly until it does not coincide with the agent's location
        self._target_location = self._agent_location
        while np.array_equal(self._target_location, self._agent_location):
            self._target_location = self.np_random.integers(0, self.size, size=2)
            
        # for testing
        # self._target_location = np.array([3,1])

        # clean the render collection and add the initial frame
        if self.render_mode != "hide":
            self.renderer.reset()
            self.renderer.render_step()

        observation = self._get_obs()
        info = self._get_info()
        return (observation, info) if return_info else observation
    
    def step(self, action):
        # Map the action (element of {0,1,2,3}) to the direction we walk in
        direction = self._action_to_direction[action]
        # We use `np.clip` to make sure we don't leave the grid
        self._agent_location = np.clip(
            self._agent_location + direction, 0, self.size - 1
        )
        # An episode is done if the agent has reached the target
        done = np.array_equal(self._agent_location, self._target_location)
        reward = 1 if done else 0  # Binary sparse rewards
        observation = self._get_obs()
        info = self._get_info()

        # add a frame to the render collection
        if self.render_mode != "hide":
            self.renderer.render_step()

        return observation, reward, done, info
    
    def render(self):
        # Just return the list of render frames collected by the Renderer.
        return self.renderer.get_renders()

    def _render_frame(self, mode: str):
        # This will be the function called by the Renderer to collect a single frame.
        assert mode is not None  # The renderer will not call this function with no-rendering.
    
        import pygame # avoid global pygame dependency. This method is not called with no-render.
    
        canvas = pygame.Surface((self.window_size, self.window_size))
        canvas.fill((255, 255, 255))
        pix_square_size = (
            self.window_size / self.size
        )  # The size of a single grid square in pixels

        # First we draw the target => a red rect
        pygame.draw.rect(
            canvas,
            (255, 0, 0),
            pygame.Rect(
                pix_square_size * self._target_location,
                (pix_square_size, pix_square_size),
            ),
        )
        # Now we draw the agent => a blue circle
        pygame.draw.circle(
            canvas,
            (0, 0, 255),
            (self._agent_location + 0.5) * pix_square_size,
            pix_square_size / 3,
        )

        # Finally, add some gridlines
        for x in range(self.size + 1):
            pygame.draw.line(
                canvas,
                0,
                (0, pix_square_size * x),
                (self.window_size, pix_square_size * x),
                width=3,
            )
            pygame.draw.line(
                canvas,
                0,
                (pix_square_size * x, 0),
                (pix_square_size * x, self.window_size),
                width=3,
            )

        if mode == "human":
            assert self.window is not None
            # The following line copies our drawings from `canvas` to the visible window
            self.window.blit(canvas, canvas.get_rect())
            pygame.event.pump()
            pygame.display.update()

            # We need to ensure that human-rendering occurs at the predefined framerate.
            # The following line will automatically add a delay to keep the framerate stable.
            self.clock.tick(self.metadata["render_fps"])
        else:  # rgb_array or single_rgb_array
            return np.transpose(
                np.array(pygame.surfarray.pixels3d(canvas)), axes=(1, 0, 2)
            )
        
    def close(self):
        if self.window is not None:
            import pygame 
            
            pygame.display.quit()
            pygame.quit()        

In [3]:
## test custom enviroment using random policy

# init env to test
env = GridTreasureEnv(render_mode="human",size=5)
# init action
env.action_space.seed(42)
# init reset
observation, info = env.reset(seed=42, return_info=True)
# random walk
for t in range(1000):
    observation, reward, done, info = env.step(env.action_space.sample())
    
    if t%100 == 0:
        print("="*10,"\t timestep is {}".format(t))
        print("Observation: {}".format(observation))
        print("Info: {}".format(info))

    if done:
        observation, info = env.reset(return_info=True)
        if t%100 != 0:
            print("="*10,"\t timestep is {}".format(t))
            print("Observation: {}".format(observation))
            print("Info: {}".format(info))
        print("~~~~done!~~~~")
        break

env.close()

========== 	 timestep is 0
Observation: {'agent': array([1, 3]), 'target': array([3, 2])}
Info: {'distance': 3.0}
========== 	 timestep is 100
Observation: {'agent': array([1, 1]), 'target': array([3, 2])}
Info: {'distance': 3.0}
========== 	 timestep is 118
Observation: {'agent': array([2, 4]), 'target': array([0, 3])}
Info: {'distance': 3.0}
~~~~done!~~~~


## Step3: Define Data Structure for Markov Decision Progress

$一个\mathrm{MDP}由五个部分组成：<S,A,P,R,\gamma>,外加智能策略\pi$

$其中：S = \{s_1,s_2,..,s_n\}为状态集合，A = \{a_1,a_2,..,a_m\}为动作集合，\pi是S,A上的条件分布：\pi(a_k|s_i) = P(A_{t}=a_k|S_t=a_i)$

$\Rightarrow 对于确定性的最佳策略\pi^*, \pi^*(s_i) = \mathrm{arg}\max\limits_{a_k\in A} \pi(a_k|s_i)$

$P为状态转移概率，具体定义有两种：$

$$\begin{equation}\begin{aligned}
&P_{s_i,s_j}^{a_k} = P(S_{t+1}=s_j | S_t=s_i,A_t=a_k) \Rightarrow 状态s_i下执行动作a_k后，转移到状态s_j的概率\\
&P_{s_i,s_j}^{\pi} = \sum\limits_{a_k\in A} \pi(a_k|s_i) \cdot P_{s_i,s_j}^{a_k} \Rightarrow 状态s_i下执行策略\pi后，转移到状态s_j的概率
\end{aligned}\end{equation}
$$

$R为立即回报，具体定义有两种：$

$$\begin{equation}\begin{aligned}
&R_{s_i}^{a_k} = E[R_{t+1}|S_t=s_i,A_t=a_k] \Rightarrow 状态s_i下执行动作a_k后，得到的立即回报\\
&R_{s_i}^{\pi} = \sum\limits_{a_k\in A} \pi(a_k|s_i)  \cdot R_{s_i}^{a_k} \Rightarrow 状态s_i下执行策略\pi后，得到的立即回报
\end{aligned}\end{equation}
$$

$\gamma为衰减因子(\mathrm{Discount}\; \mathrm{Factor})\in [0,1]，在计算未来累计回报G时，用来表示未来时刻的立即回报的权重衰减率$

$$\begin{equation}\begin{aligned}
& G_t = R_{t+1} + \gamma R_{t+2} + \gamma^2 R_{t+3} + ...  = \sum\limits_{k=0}^{\infty} \gamma^kR_{t+k+1} \Rightarrow 描述t时刻之后的未来累计的\gamma指数衰减加权回报\\
\end{aligned}\end{equation}
$$

$\Rightarrow 当\gamma = 0时，表示无需考虑未来的回报，只需考虑当下的立即回报；当\gamma = 1时，表示任意遥远的未来回报同当下的立即汇报同等重要$

In [4]:
class Policy():
    def __init__(self,numS,numA,uniform=False):
        self.numS = numS
        self.numA = numA
        self.pi = np.zeros(shape=(numS,numA),dtype=np.float32)
        if uniform:
            self.to_uniform()
    def update(self,si,ak,val):
        self.pi[si][ak] = val
    def get(self,si,ak):
        return self.pi[si][ak]
    def getBestAction(self,si,all_actions=False):
        if all_actions:
            max_val = np.max(self.pi[si,:])
            best_action_list = []
            for a,v in enumerate(self.pi[si,:]):
                if v == max_val:
                    best_action_list.append(a)
            return np.array(best_action_list)
        return np.argmax(self.pi[si,:])
    def to_uniform(self):
        self.pi = np.ones(shape=(self.numS,self.numA),dtype=np.float32) / self.numA

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
class ProbTransformer():
    def __init__(self,numS,numA):
        self.numS = numS
        self.numA = numA
        self.p = np.zeros(shape=(numS,numS,numA),dtype=np.float32)
    def update(self,si,sj,ak,val):
        self.p[si][sj][ak] = val
    def getByAction(self,si,sj,ak):
        return self.p[si][sj][ak]
    def getByPolicy(self,si,sj,pi:Policy):
        return np.sum(
            [ pi.get(si,ak) * self.p[si][sj][ak] for ak in range(self.numA) ]
        )

In [6]:
class Reward():
    def __init__(self,numS,numA):
        self.numS = numS
        self.numA = numA
        self.r = np.zeros(shape=(numS,numA),dtype=np.float32)
    def update(self,si,ak,val):
        self.r[si][ak] = val
    def getByAction(self,si,ak):
        return self.r[si][ak]
    def getByPolicy(self,si,pi:Policy):
        return np.sum(
            [ pi.get(si,ak) * self.r[si][ak] for ak in range(self.numA) ]
        )

In [7]:
class MDP: # <S,A,P,R,γ> + π
    def __init__(self,numS,numA,pi:Policy,gamma=1):
        self.numS = numS
        self.numA = numA
        self.P = ProbTransformer(numS,numA)
        self.R = Reward(numS,numA)
        self.gamma = gamma
        self.pi = pi

## Step4: Define Data Structure for Bellman Equation

$贝尔曼方程是求解\mathrm{MDP}最优策略的基本理论，因此也是强化学习的理论基石$

$贝尔曼方程包含期望方程和最优方程两种形式：$

$\Rightarrow 1. 贝尔曼期望方程：描述了当前值函数（包括状态值函数V_{\pi}(s_i)和行为值函数Q_{\pi}(s_i,a_k)）的递推关系，具体定义如下：$

$$\begin{equation}\begin{aligned}
&Q_{\pi}(s_i,a_k) = E_{\pi}[G_t|S_t = s_i,A_t=a_k]\Rightarrow 状态s_i下执行动作a_k后，能得到的期望未来累计回报\\
&V_{\pi}(s_i) = E_{\pi}[G_t|S_t = s_i] = \sum\limits_{a_k\in A} \pi(a_k|s_i)\cdot Q_{\pi}(s_i,a_k)\Rightarrow 状态s_i下执行策略\pi后，能得到的期望未来累计回报\\
\end{aligned}\end{equation}
$$

$\quad\quad *\; 行为值函数Q_{\pi}(s_i,a_k)的递推方程推导如下：$

$$\begin{equation}\begin{aligned}
Q_{\pi}(s_i,a_k) 
&= E_{\pi}[G_t|S_t = s_i,A_t=a_k]\\
&= E_{\pi}[R_{t+1}+\gamma G_{t+1}|S_t = s_i,A_t=a_k]\\
&= E_{\pi}[R_{t+1}|S_t = s_i,A_t=a_k] + \gamma E_{\pi}[ G_{t+1}|S_t = s_i,A_t=a_k]\\
&= R_{s_i}^{a_k} + \gamma \sum\limits_{s_j\in S} P(S_{t+1}=s_j | S_t=s_i,A_t=a_k) \cdot E_{\pi}[ G_{t+1}|S_{t+1} = s_j;S_t = s_i,A_t=a_k]\\
&= R_{s_i}^{a_k} + \gamma \sum\limits_{s_j\in S} P_{s_i,s_j}^{a_k} \cdot V_{\pi}(s_j)\\
&= R_{s_i}^{a_k} + \gamma \sum\limits_{s_j\in S} P_{s_i,s_j}^{a_k} \cdot [\sum\limits_{a_l\in A} \pi(a_l|s_j)\cdot Q_{\pi}(s_j,a_l)]
\end{aligned}\end{equation}
$$

$\quad\quad *\; 状态值函数V_{\pi}(s_i)的递推方程推导如下：$

$$\begin{equation}\begin{aligned}
V_{\pi}(s_i) 
&= \sum\limits_{a_k\in A} \pi(a_k|s_i)\cdot Q_{\pi}(s_i,a_k)\\
&= \sum\limits_{a_k\in A} \pi(a_k|s_i)\cdot [R_{s_i}^{a_k} + \gamma \sum\limits_{s_j\in S} P_{s_i,s_j}^{a_k} \cdot V_{\pi}(s_j)]\\
\end{aligned}\end{equation}
$$

$\Rightarrow 2. 贝尔曼最优方程：描述了当前最优值函数（包括状态最优值函数V^*(s_i)和行为最优值函数Q^*(s_i,a_k)）的递推关系，具体定义如下：$

$$\begin{equation}\begin{aligned}
&Q^*(s_i,a_k) = \max\limits_{\pi} Q_{\pi}(s_i,a_k) = Q_{\pi^*}(s_i,a_k)  \Rightarrow 所有策略下值最优的行为值函数，亦对应最优策略\\
&V^*(s_i) = \max\limits_{\pi} V_{\pi}(s_i) = \max\limits_{a_k\in A} Q^*(s_i,a_k) = V_{\pi^*}(s_i) \Rightarrow 所有策略下值最优的状态值函数，亦对应最优策略和行为最优值函数\\
\end{aligned}\end{equation}
$$

$\quad\quad *\; 行为最优值函数Q^*(s_i,a_k)的递推方程推导如下：$

$$\begin{equation}\begin{aligned}
Q^*(s_i,a_k) 
&= R_{s_i}^{a_k} + \gamma \sum\limits_{s_j\in S} P_{s_i,s_j}^{a_k} \cdot V^*(s_j)\\
&= R_{s_i}^{a_k} + \gamma \sum\limits_{s_j\in S} P_{s_i,s_j}^{a_k} \cdot \max\limits_{a_l\in A} Q^*(s_j,a_l)\\
\end{aligned}\end{equation}
$$

$\quad\quad *\; 状态最优值函数V^*(s_i)的递推方程推导如下：$

$$\begin{equation}\begin{aligned}
V^*(s_i)
&= \max\limits_{a_k\in A} Q^*(s_i,a_k) \\
&= \max\limits_{a_k\in A} [R_{s_i}^{a_k} + \gamma \sum\limits_{s_j\in S} P_{s_i,s_j}^{a_k} \cdot V^*(s_j)]\\
\end{aligned}\end{equation}
$$

$\quad\quad *\; 最优策略\pi^*的求解方程如下：$

$$\begin{equation}
\pi^*(a_k|s_i) = 
\begin{cases}
1, & a_k= \mathrm{arg}\max\limits_{a_l\in A} Q^*(s_i,a_l) \\
0, & otherwise
\end{cases}\\
\end{equation}
$$

$\quad\quad \Rightarrow 若状态s_i下存在多个使得行为值函数最优的动作集合B = \{a_{k_1},a_{k_2},..,a_{k_{\beta}}\}, 则：\pi^*(a_{k}|s_i) =  \begin{cases}
\cfrac{1}{\beta}, & a_{k} \in B \\
0, & otherwise
\end{cases}
$

$\quad\quad \Rightarrow 对于任何\mathrm{MDP}问题，总存在一个\textbf{确定性}的最优策略\pi^*，其亦对应状态最优值函数V^*(s_i)和行为最优值函数Q^*(s_i,a_k)$

In [19]:
class BE: # Bellman Equation for MDP model
    def __init__(self,mdp:MDP):
        self.mdp = mdp
        self.Q = np.zeros(shape=(mdp.numS,mdp.numA),dtype=np.float32)
        self.V = np.zeros(shape=(mdp.numS,),dtype=np.float32)
        self.bQ = np.zeros(shape=(mdp.numS,mdp.numA),dtype=np.float32)
        self.bV = np.zeros(shape=(mdp.numS,),dtype=np.float32)
    def updateQ(self,si,ak):
        old_val = self.Q[si][ak]
        new_val = self.mdp.R.getByAction(si,ak) + self.mdp.gamma * np.sum([
            self.mdp.P.getByAction(si,sj,ak) * np.sum([
                self.mdp.pi.get(sj,al) * self.Q[sj][al] for al in range(self.mdp.numA)
            ]) for sj in range(self.mdp.numS)
        ])
        
        self.Q[si][ak] = new_val
        return old_val
    def updateV(self,si):
        old_val = self.V[si]
        new_val = np.sum([
            self.mdp.pi.get(si,ak) * (
                self.mdp.R.getByAction(si,ak) + self.mdp.gamma * np.sum([
                    self.mdp.P.getByAction(si,sj,ak) * self.V[sj] for sj in range(self.mdp.numS)
                ])
            ) for ak in range(self.mdp.numA)
        ])
        
        self.V[si] = new_val
        return old_val
    def updateBestQ(self,si,ak):
        old_val = self.bQ[si][ak]
        new_val = self.mdp.R.getByAction(si,ak) + self.mdp.gamma * np.sum([
            self.mdp.P.getByAction(si,sj,ak) * np.max(self.bQ[sj,:]) for sj in range(self.mdp.numS)
        ])
        
        self.bQ[si][ak] = new_val
        return old_val
    def updateBestV(self,si):
        old_val = self.bV[si]
        new_val = np.max([
            self.mdp.R.getByAction(si,ak) + self.mdp.gamma * np.sum([
                    self.mdp.P.getByAction(si,sj,ak) * self.bV[sj] for sj in range(self.mdp.numS)
                ])
            for ak in range(self.mdp.numA)
        ])
        
        self.bV[si] = new_val
        return old_val
    def updatePolicyWithV(self,si):
        val_list = np.array([
            self.mdp.R.getByAction(si,al) + self.mdp.gamma * np.sum([
                self.mdp.P.getByAction(si,sj,al) * self.V[sj] for sj in range(self.mdp.numS)
            ])
            for al in range(self.mdp.numA)
        ])
        B = []
        max_val = np.max(val_list)
        
        for a,v in enumerate(val_list):
            if v == max_val:
                B.append(a)
        
        for ak in range(self.mdp.numA):
            if ak in B:
                self.mdp.pi.update(si,ak,1./len(B))
            else:
                self.mdp.pi.update(si,ak,0.0)
    def updatePolicyWithbV(self,si):
        val_list = np.array([
            self.mdp.R.getByAction(si,al) + self.mdp.gamma * np.sum([
                self.mdp.P.getByAction(si,sj,al) * self.bV[sj] for sj in range(self.mdp.numS)
            ])
            for al in range(self.mdp.numA)
        ])
        B = []
        max_val = np.max(val_list)
        
        for a,v in enumerate(val_list):
            if v == max_val:
                B.append(a)
        
        for ak in range(self.mdp.numA):
            if ak in B:
                self.mdp.pi.update(si,ak,1./len(B))
            else:
                self.mdp.pi.update(si,ak,0.0)
    def getQ(self,si,ak):
        return self.Q[si][ak]
    def getV(self,si):
        return self.V[si]
    def getbQ(self,si,ak):
        return self.bQ[si][ak]
    def getbV(self,si):
        return self.bV[si]

## Step5: Build Modeling Policy using Policy-Iterative Dynamic Programming Algorithm

In [14]:
## init data structure
seed = 42
env = GridTreasureEnv(render_mode="hide",size=5)
numS = env.numS
numA = env.numA

policy = Policy(numS,numA,uniform=True)
mdp = MDP(numS,numA,policy,gamma=1)
be = BE(mdp)

# random policy at first
policy.pi

array([[0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25]], dtype=float32)

In [15]:
## build policy-iterative dynamic programming algorithm
def policyDP(env,mdp,be,seed=42,threshold=1e-5,log=False):
    observation = env.reset(seed=seed, return_info=False)
    target_loc = observation["target"]
    agent_loc = observation["agent"]
    
    # init prob transformer P[si->sj][ak] = 1 if si+ak => sj else 0
    # except when si == target_state, P[si->sj][ak] = 1 if sj == si else 0
    for si in range(mdp.numS):
        for ak in range(mdp.numA):
            loci = env.state_to_loc(si)
            if (target_loc == loci).all():
                mdp.P.update(si,si,ak,1.)
            else:
                dirtk = env._action_to_direction[ak]
                loci = env.state_to_loc(si)
                locj = np.clip(loci+dirtk,0,env.size-1)
                mdp.P.update(si,env.loc_to_state(locj),ak,1.)
                    
    # init reward R[si][ak] = 0 if si == target_loc else -1
    for si in range(mdp.numS):
        for ak in range(mdp.numA):
            loci = env.state_to_loc(si)
            if (target_loc == loci).all():
                mdp.R.update(si,ak,0)
            else:
                mdp.R.update(si,ak,-1)
    
    
    # policy evalute-improvement
    eval_epoch_list = []
    while True: # update policy until it's converged
        # policy evaluate
        eval_epoch = 0
        while True: # update V until it's converged
            delta = 0
            for si in range(mdp.numS): 
                old_val = be.updateV(si)
                new_val = be.getV(si)
                delta = max(delta,np.abs(new_val-old_val))   
            eval_epoch += 1
            if delta < threshold:
                break
            
        if log:
            eval_epoch_list.append(eval_epoch)
            print("="*10)
            print("epoch {} used {} iterations to converge V".format(len(eval_epoch_list),eval_epoch_list[-1]))
            print("The current converged V is:\n",be.V)
        
                
        # policy improvement
        changed = False
        for si in range(mdp.numS): 
            old_best_action = mdp.pi.getBestAction(si)
            be.updatePolicyWithV(si)
            new_best_actions = mdp.pi.getBestAction(si,all_actions=True)
            if old_best_action not in new_best_actions:
                changed = True
        
        if log:
            print("The curent best policy is:\n",mdp.pi.pi)
                
        if not changed: 
            if log:
                print("~"*10)
                print("The best policy has converged!")
                print("~"*10)
            break


In [16]:
# using policy-iterative dynamic programming to get best policy
policyDP(env,mdp,be,seed=seed,log=True)
print("DP done!")

epoch 1 used 293 iterations to converge V
The current converged V is:
 [-47.408867 -44.136166 -38.454384 -32.7726   -31.499882 -46.681606
 -42.545273 -34.454407 -24.363544 -26.22718  -46.090706 -40.90892
 -28.454433   0.       -18.81812  -46.681614 -42.54528  -34.454414
 -24.36355  -26.227184 -47.408882 -44.13618  -38.4544   -32.77261
 -31.499893]
The curent best policy is:
 [[1.   0.   0.   0.  ]
 [1.   0.   0.   0.  ]
 [1.   0.   0.   0.  ]
 [0.   1.   0.   0.  ]
 [0.   1.   0.   0.  ]
 [1.   0.   0.   0.  ]
 [1.   0.   0.   0.  ]
 [1.   0.   0.   0.  ]
 [0.   1.   0.   0.  ]
 [0.   1.   0.   0.  ]
 [1.   0.   0.   0.  ]
 [1.   0.   0.   0.  ]
 [1.   0.   0.   0.  ]
 [0.25 0.25 0.25 0.25]
 [0.   0.   1.   0.  ]
 [1.   0.   0.   0.  ]
 [1.   0.   0.   0.  ]
 [1.   0.   0.   0.  ]
 [0.   0.   0.   1.  ]
 [0.   0.   0.   1.  ]
 [1.   0.   0.   0.  ]
 [1.   0.   0.   0.  ]
 [1.   0.   0.   0.  ]
 [0.   0.   0.   1.  ]
 [0.   0.   0.   1.  ]]
epoch 2 used 6 iterations to converge V
The cu

In [17]:
# show the best policy on grid world
import pandas as pd
policy_grid = pd.DataFrame(np.zeros(shape=(env.size,env.size)))
action_map = {
    0:">", # right
    1:"v", # up
    2:"<", # left
    3:"^"  # down
}
for si in range(mdp.numS):
    loc = env.state_to_loc(si)
    s = ""
    for a in policy.getBestAction(si,all_actions=True):
        s += action_map[a]
    policy_grid.loc[loc[1],loc[0]] = s
policy_grid

,0,1,2,3,4
0,>v,>v,>v,v,v<
1,>v,>v,>v,v,v<
2,>,>,>,>v<^,<
3,>^,>^,>^,^,<^
4,>^,>^,>^,^,<^


In [18]:
# test best policy on real environment
env = GridTreasureEnv(render_mode="human",size=5)
# init reset
observation, info = env.reset(seed=seed, return_info=True)
# walk with best policy
for t in range(100):
    agent_loc = observation["agent"]
    best_actions = policy.getBestAction(env.loc_to_state(agent_loc),all_actions=True)
    best_action = np.random.choice(best_actions,size=1).item()
    observation, reward, done, info = env.step(best_action)
    
    # log time step
    print("="*10,"\t timestep is {}".format(t))
    print("Observation: {}".format(observation))
    print("Info: {}".format(info))
        

    if done:
        observation, info = env.reset(return_info=True)
        print("~~~~done!~~~~")
        break

env.close()

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


========== 	 timestep is 0
Observation: {'agent': array([1, 3]), 'target': array([3, 2])}
Info: {'distance': 3.0}
========== 	 timestep is 1
Observation: {'agent': array([2, 3]), 'target': array([3, 2])}
Info: {'distance': 2.0}
========== 	 timestep is 2
Observation: {'agent': array([2, 2]), 'target': array([3, 2])}
Info: {'distance': 1.0}
========== 	 timestep is 3
Observation: {'agent': array([3, 2]), 'target': array([3, 2])}
Info: {'distance': 0.0}
~~~~done!~~~~


## Step6: Build Modeling Policy using Value-Iterative Dynamic Programming Algorithm

In [30]:
## init data structure
seed = 42
env = GridTreasureEnv(render_mode="hide",size=5)
numS = env.numS
numA = env.numA

policy = Policy(numS,numA,uniform=True)
mdp = MDP(numS,numA,policy,gamma=1)
be = BE(mdp)

# random policy at first
policy.pi

array([[0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25]], dtype=float32)

In [31]:
## build value-iterative dynamic programming algorithm
def valueDP(env,mdp,be,seed=42,threshold=1e-5,log=False):
    observation = env.reset(seed=seed, return_info=False)
    target_loc = observation["target"]
    agent_loc = observation["agent"]
    
    # init prob transformer P[si->sj][ak] = 1 if si+ak => sj else 0
    # except when si == target_state, P[si->sj][ak] = 1 if sj == si else 0
    for si in range(mdp.numS):
        for ak in range(mdp.numA):
            loci = env.state_to_loc(si)
            if (target_loc == loci).all():
                mdp.P.update(si,si,ak,1.)
            else:
                dirtk = env._action_to_direction[ak]
                loci = env.state_to_loc(si)
                locj = np.clip(loci+dirtk,0,env.size-1)
                mdp.P.update(si,env.loc_to_state(locj),ak,1.)
                    
    # init reward R[si][ak] = 0 if si == target_loc else -1
    for si in range(mdp.numS):
        for ak in range(mdp.numA):
            loci = env.state_to_loc(si)
            if (target_loc == loci).all():
                mdp.R.update(si,ak,0)
            else:
                mdp.R.update(si,ak,-1)
    
    
    # policy evalute-improvement
    # update policy when best V is converged
    
    # policy evaluate
    eval_epoch = 0
    while True: # update best V until it's converged
        delta = 0
        for si in range(mdp.numS): 
            old_val = be.updateBestV(si)
            new_val = be.getbV(si)
            delta = max(delta,np.abs(new_val-old_val))   
            
        if log:
            eval_epoch += 1
            print("="*10)
            print("epoch: {} | The current bV is:\n".format(eval_epoch),be.bV)
        
        
        if delta < threshold:
            break


    # policy improvement with best V
    for si in range(mdp.numS): 
        be.updatePolicyWithbV(si)

    if log:
        print("~"*10)
        print("The best policy is:\n",mdp.pi.pi)
        print("~"*10)


In [32]:
# using value-iterative dynamic programming to get best policy
valueDP(env,mdp,be,seed=seed,log=True)
print("DP done!")

epoch: 1 | The current bV is:
 [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.  0. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1.]
epoch: 2 | The current bV is:
 [-2. -2. -2. -2. -2. -2. -2. -2. -1. -2. -2. -2. -1.  0. -1. -2. -2. -2.
 -1. -2. -2. -2. -2. -2. -2.]
epoch: 3 | The current bV is:
 [-3. -3. -3. -2. -3. -3. -3. -2. -1. -2. -3. -2. -1.  0. -1. -3. -3. -2.
 -1. -2. -3. -3. -3. -2. -3.]
epoch: 4 | The current bV is:
 [-4. -4. -3. -2. -3. -4. -3. -2. -1. -2. -3. -2. -1.  0. -1. -4. -3. -2.
 -1. -2. -4. -4. -3. -2. -3.]
epoch: 5 | The current bV is:
 [-5. -4. -3. -2. -3. -4. -3. -2. -1. -2. -3. -2. -1.  0. -1. -4. -3. -2.
 -1. -2. -5. -4. -3. -2. -3.]
epoch: 6 | The current bV is:
 [-5. -4. -3. -2. -3. -4. -3. -2. -1. -2. -3. -2. -1.  0. -1. -4. -3. -2.
 -1. -2. -5. -4. -3. -2. -3.]
~~~~~~~~~~
The best policy is:
 [[0.5  0.5  0.   0.  ]
 [0.5  0.5  0.   0.  ]
 [0.5  0.5  0.   0.  ]
 [0.   1.   0.   0.  ]
 [0.   0.5  0.5  0.  ]
 [0.5  0.5  0.   0.  ]
 [0.5  0.5  0.   0.  ]
 [0

In [33]:
# show the best policy on grid world
import pandas as pd
policy_grid = pd.DataFrame(np.zeros(shape=(env.size,env.size)))
action_map = {
    0:">", # right
    1:"v", # up
    2:"<", # left
    3:"^"  # down
}
for si in range(mdp.numS):
    loc = env.state_to_loc(si)
    s = ""
    for a in policy.getBestAction(si,all_actions=True):
        s += action_map[a]
    policy_grid.loc[loc[1],loc[0]] = s
policy_grid

,0,1,2,3,4
0,>v,>v,>v,v,v<
1,>v,>v,>v,v,v<
2,>,>,>,>v<^,<
3,>^,>^,>^,^,<^
4,>^,>^,>^,^,<^


In [34]:
# test best policy on real environment
env = GridTreasureEnv(render_mode="human",size=5)
# init reset
observation, info = env.reset(seed=seed, return_info=True)
# walk with best policy
for t in range(100):
    agent_loc = observation["agent"]
    best_actions = policy.getBestAction(env.loc_to_state(agent_loc),all_actions=True)
    best_action = np.random.choice(best_actions,size=1).item()
    observation, reward, done, info = env.step(best_action)
    
    # log time step
    print("="*10,"\t timestep is {}".format(t))
    print("Observation: {}".format(observation))
    print("Info: {}".format(info))
        

    if done:
        observation, info = env.reset(return_info=True)
        print("~~~~done!~~~~")
        break

env.close()

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


========== 	 timestep is 0
Observation: {'agent': array([0, 2]), 'target': array([3, 2])}
Info: {'distance': 3.0}
========== 	 timestep is 1
Observation: {'agent': array([1, 2]), 'target': array([3, 2])}
Info: {'distance': 2.0}
========== 	 timestep is 2
Observation: {'agent': array([2, 2]), 'target': array([3, 2])}
Info: {'distance': 1.0}
========== 	 timestep is 3
Observation: {'agent': array([3, 2]), 'target': array([3, 2])}
Info: {'distance': 0.0}
~~~~done!~~~~


$\Rightarrow 综上所述：$

$\mathrm{MDP}过程的求解方法都是基于贝尔曼方程，而对于模型已知的\mathrm{MDP}，可以直接利用动态规划算法解贝尔曼方程$

$根据使用的贝尔曼方程的形式的不同，动态规划算法又分为策略迭代和值迭代两种$

$①策略迭代的动态规划算法使用的是贝尔曼期望方程，算法整体分为两步：\\step1. 策略评估，即在当前策略下，通过状态值函数的迭代方程，迭代计算，直到状态值函数收敛\\step2. 策略改进，即根据上一步评估的状态值函数，更新最佳策略，重复执行step1,2，直到最佳策略收敛\\\Rightarrow 由于两步均需要迭代，因此计算开销比较大，效率较低$

$②值迭代的动态规划算法使用的是贝尔曼最优方程，算法也分为两步：\\step1.策略评估+改进，即在当前策略下，通过状态最优值函数的迭代方程，迭代计算，直到状态最优值函数收敛 \\\;\Rightarrow 看似是在评估，实际上最优值函数的更新就是在改进策略(选择当前使得值函数最大的行为所对应的值)，因此当其收敛时，最优策略亦收敛\\step2. 根据上一步评估的状态最优值函数，更新最优策略即可\\\Rightarrow 仅第一步需要迭代，而且由于评估的同时亦在更新，因此迭代次数明显减小，效率较高$